# Clean Commodity Data

In [ ]:
# Imports

import pandas as pd
import numpy as np

In [ ]:
# Read commodities from file

commodities_df = pd.read_csv('../data/commodities/d16_h02_commodities.csv', index_col = 0)

print(commodities_df.shape)

commodities_df.head()

In [ ]:
# Drop columns 'auction_id' + 'time_left'

commodities_df.drop(['auction_id', 'time_left'], axis = 1, inplace = True)

commodities_df.head()

In [ ]:
# Check all rows of a specific item

commodities_df[commodities_df['item_id'] == 173033]

In [ ]:
# As there is no way to get information about the sold commodity auctions I will use the weighted (quantity) median to pick a reasonable unit price.
# This will be less sensitive to outliers and less likely to be affected by sudden spikes or dips in the price of a particular item.

def weighted_median(item_df):
   sorted_df = item_df.sort_values('unit_price') # sort values in ascending order by unit_price
   cum_weights = sorted_df['quantity'].cumsum() # calculate the cumulative sum of quantities
   total_weight = cum_weights.iloc[-1] # get the total weight
   if total_weight % 2 == 1: # if the total weight is odd
      median_index = cum_weights.searchsorted(total_weight / 2) # find the index of the median
      return sorted_df.iloc[median_index]['unit_price'] # return the corresponding unit_price value
   else: # if the total weight is even
      median_index = cum_weights.searchsorted(total_weight / 2, side = 'right') # find the index of the right median
      median_values = sorted_df.iloc[median_index - 1: median_index]['unit_price'] # get the two median values
      # If the mean of the two median values is NaN, return the median unit price (unweighted)
      return item_df['unit_price'].median() if pd.isna(median_values.mean()) else median_values.mean()
     
price_weighted_median = commodities_df.groupby('item_id').apply(lambda x: weighted_median(x)).reset_index(name = 'weighted')

price_weighted_median['weighted'] = price_weighted_median['weighted'].astype(int)

price_weighted_median

In [ ]:
# Group by item_id and sum quantity

commodities_df = commodities_df.groupby('item_id').agg({'unit_price': 'min', 'timestamp': 'first', 'quantity': 'sum'})

commodities_df

In [ ]:
# Merge commodities + weighted median price

commodities_df = pd.merge(commodities_df, price_weighted_median, left_on = 'item_id', right_on = 'item_id')

commodities_df = commodities_df[['item_id', 'quantity', 'unit_price', 'weighted', 'timestamp']]

commodities_df

In [ ]:
# Check for remaining NaN

commodities_df.isna().sum()